# 7 Best Practices, Testing and Performance

Advanced bits marked with ⚠ might be more for reference later.

## PEP8 -- Style Guide for Python Code

Intendation is significant in Python, preventing the worst mess. The remaining stylistic questions, like naming conventions, used in the standard library are summarized in the [Python Enhancement Proposal 8](https://www.python.org/dev/peps/pep-0008/). Most Python programmers adhere to this style guide (Google has their [own](https://google.github.io/styleguide/pyguide.html) which is similar and also popular), making it easy to read someone elses code. 

⚠ Some editors and tools like `autopep8` help check for violations. Other tools, like [yapf](https://github.com/google/yapf), go further and reformat your code, e.g.

```python
def forces(t,
                X,      N):
    """Calculate forces from neighbouring cells"""
    for i, x in enumerate(X):
        r = X[N[i]] - x
        norm_r = np.minimum             (np.linalg.norm(r,


        
        axis=1), r_max)
        norm_F = 2*(r_min -       norm_r)*(norm_r - r_max) - (norm_r - r_max)**2
        F[i] = np.sum(r*(
                                norm_F/
                    norm_r)[:, None], axis=0)
    return F.ravel()

```

becomes:

```python
def forces(t, X, N):
    """Calculate forces from neighbouring cells"""
    for i, x in enumerate(X):
        r = X[N[i]] - x
        norm_r = np.minimum(np.linalg.norm(r, axis=1), r_max)
        norm_F = 2 * (r_min - norm_r) * (norm_r - r_max) - (norm_r - r_max)**2
        F[i] = np.sum(r * (norm_F / norm_r)[:, None], axis=0)
    return F.ravel()

```

⚠ Compilation would prevent many erros that are only detected at run-time in uncompiled languages like Python. But some Editors and tools like [pyflakes](https://pypi.python.org/pypi/pyflakes) and [pylint](https://www.pylint.org/) help with static code analysis to prevent the worst errors.

## PEP 20 -- The Zen of Python

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## Structuring Code

* Don't repeat yourself (and others) to keep your programs small

* Structure code into functions and files. You can import functions and data from files (so-called modules):

```python
# File module/example.py
x = 'Example data'

def example_function():
    pass

if __name__ == '__main__':
    whatever()
    
# File in .
from module import example

example.x
```

## Documentation

* Document why and not what (what states the code)

* Commit early and often to keep track of why you change things

* False (usually out-dated) documentation is worse than none

* Turn questions (yours or others) into documentation

* Write [self-documenting code](https://testing.googleblog.com/2017/07/code-health-to-comment-or-not-to-comment.html), e.g. (from real-life C++) change

```C++
if ((j != k) and (d_type[d_lattice->d_cell_id[j]] == mesenchyme)
        and (d_type[d_lattice->d_cell_id[k]] == mesenchyme)
        and (dist < r_link)
        and (fabs(r.w/(d_X[d_lattice->d_cell_id[j]].w + 
             d_X[d_lattice->d_cell_id[k]].w)) > 0.2)) {
    d_link[i].a = d_lattice->d_cell_id[j];
    d_link[i].b = d_lattice->d_cell_id[k];
}
```

to
```C++
auto both_mesenchyme = (d_type[d_lattice->d_cell_id[j]] == mesenchyme)
    and (d_type[d_lattice->d_cell_id[k]] == mesenchyme);
auto along_w = fabs(r.w/(d_X[d_lattice->d_cell_id[j]].w + 
                    d_X[d_lattice->d_cell_id[k]].w)) > 0.2;
if (both_mesenchyme and (dist < r_link) and along_w) {
    d_link[i].a = d_lattice->d_cell_id[j];
    d_link[i].b = d_lattice->d_cell_id[k];
}
```

* Use `docstrings`: 

```python
def documented_function():
    """This is a docstring"""
    pass
```

* ⚠ Turn documentation into code using `docopt`:

```python
"""A Docopt Example.

Usage:
  docoptest.py [flag] [--parameter <x>]
  docoptest.py (-h | --help)

Options:
  -h --help        Show this screen.
  --parameter <x>  Pass parameter.
"""
if __name__ == '__main__':
    from docopt import docopt

    args = docopt(__doc__)
    print(args)

```

Results in
```bash
$ python docoptest.py wrong usage
Usage:
  docoptest.py [flag] [--parameter <x>]
  docoptest.py (-h | --help)

$ python docoptest.py -h
A Docopt Example.

Usage:
  docoptest.py [flag] [--parameter <x>]
  docoptest.py (-h | --help)

Options:
  -h --help        Show this screen.
  --parameter <x>  Pass parameter.

$ python docoptest.py flag
{'--help': False,
 '--parameter': None,
 'flag': True}

```

## Testing

* Turn debugging `print`-statements into `assert`-statements

In [2]:
mass = -1
assert mass > 0, 'Mass cannot be negative!'

AssertionError: Mass cannot be negative!

* Make sure your code does what it is supposed to do by testing it with simple examples where you know what to expect

* Save those tests (e.g. in `if __name__ == __main__` of each module) to keep your code correct while changing parts

* ⚠ Automate testing

```python
import unittest

def square(x):
    return x*x

class TestSquare(unittest.TestCase):

    def test_square(self):
        self.assertEqual(square(2), 4)


if __name__ == '__main__':
    unittest.main()
```

* ⚠ Generate fuzzy tests using `hypothesis`:

```python
from hypothesis import given
from hypothesis.strategies import text, floats

@given(text(min_size=10), text(min_size=10), text(min_size=10))
def test_triangle_inequality(self, a, b, c):
    self.assertTrue(zipstance(a, c) <= zipstance(a, b) + zipstance(b, c))
```

* ⚠ Turn documentation into tests using `doctest`:

```python
def square(x):
    """Return the square of x.

    >>> square(2)
    0
    """
    return x*x

if __name__ == '__main__':
    import doctest
    doctest.testmod()
```

Will give

```bash
$ python doctestest.py 
**********************************************************************
File "doctestest.py", line 4, in __main__.square
Failed example:
    square(2)
Expected:
    0
Got:
    4
**********************************************************************
1 items had failures:
   1 of   1 in __main__.square
***Test Failed*** 1 failures.
```

* ⚠ Use [mutation tests](http://cosmic-ray.readthedocs.io/en/latest/) if you need to be sure that your code is correct. They replace pieces of code, e.g. `<` with `>` to find out whether your tests cover everything.

## Optimizing Performance

### Quick Tests

In [3]:
%timeit [i**2 for i in range(100000)]

20.7 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
import numpy as np

%timeit np.arange(100000)**2

356 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Real-life Example

We tried to speed-up the simulation of an N-body problem describing cells interacting in pairs via the spherical potential $U(r) = -(r - r_{min})(r - r_{max})^2$, where $r = |\vec x_j - \vec x_i|$. The resulting forces can be calculated as

```python
def forces(t, X, N):
    """Calculate forces from neighbouring cells"""
    for i, x in enumerate(X):
        r = X[N[i]] - x
        norm_r = np.minimum(np.linalg.norm(r, axis=1), r_max)
        norm_F = 2*(r_min - norm_r)*(norm_r - r_max) - (norm_r - r_max)**2
        F[i] = np.sum(r*(norm_F/norm_r)[:, None], axis=0)
    return F.ravel()
```

where `N` is a matrix giving the $k$ nearest neighbours.

#### Profiling

The `cProfile` module help identify bottlenecks when running `'command'`:


```python
import cProfile

cProfile.run('command', 'nbodystats')
```

and `pstat` allows you then to analyse the data saved in `nbodystats` (in the actual code `forces` was wrapped into a class with `__call__`):

In [8]:
import pstats

p = pstats.Stats('nbodystats')
p.strip_dirs().sort_stats('cumulative').print_stats(10);

NameError: name 'command' is not defined

#### Vectorize, 7x

While the initial function was already using `numpy` and vectors it still involves a `for`-loop that can be vectorized ... well, actually tensorized:

```python
def forces(t, X, N):
    r = X[N] - np.tile(X, (k, 1, 1)).transpose(1, 0, 2)
    norm_r = np.minimum(np.linalg.norm(r, axis=2), r_max)
    norm_F = 2*(r_min - norm_r)*(norm_r - r_max) - (norm_r - r_max)**2
    F = np.sum(r*(norm_F/norm_r)[:, None].transpose(0, 2, 1), axis=1)
    return F.ravel()
```

#### Re-use resources, 1.5x

Broadcasting gives just a "view" instead of a copy returned by `np.tile`:

```python
def forces(t, X, N):
    r = X[N] - X[:, None, :]
    norm_r = np.minimum(np.linalg.norm(r, axis=2), r_max)
    norm_F = 2*(r_min - norm_r)*(norm_r - r_max) - (norm_r - r_max)**2
    F = np.sum(r*(norm_F/norm_r)[:, None].transpose(0, 2, 1), axis=1)
    return F.ravel()
```

The standard inplace operators `+=`, `-=`, `*=`, and `/=` can speed up large numpy calculations a little, Pandas operations often have an `inplace` flag, e.g. `df.reset_index(inplace=True)`. Sometimes allocating memory with `np.empty` or `pd.DataFrame` pays off. Make sure to use the fastest data type for each column of a `pd.DataFrame`.

#### ⚠ Compile the calculation, 2x (3.5x on GTX 1060 for 10k cells)

Finally code can be compiled in various ways. We choose [Theano](http://deeplearning.net/software/theano/) because it requires little rewritting (and can run code on the GPU as well):

```python
from theano import Tensor as T
from theano import function


X = T.matrix('X', dtype='floatX')
N = T.imatrix('N')

r = X[N] - X[:, None, :]
norm_r = T.minimum(r.norm(2, axis=2), 1)
norm_F = 2*(0.5 - norm_r)*(norm_r - 1) - (norm_r - 1)**2
F = T.sum(r*(norm_F/norm_r).dimshuffle(0, 1, 'x'), axis=1)
f = function([X, N], F.ravel(), allow_input_downcast=True)

def forces(t, X, N):
    return f(X.reshape(-1, 3), self.N)
```

#### Aftermath

In [ ]:
p = pstats.Stats('theanostats')
p.strip_dirs().sort_stats('cumulative').print_stats(10);

The file `function_module.py` belongs to `Theano`, there is likely not much to be gained. Still most time is spent calculating the derivatives, but allocating the `DataFrame` for the results could shave another 17% (35% when using a GPU) off in the best case. This would result in a ~50x (100x) faster code than the initial version.

This prototype allowed us to simulate a few hundred interacting cells in seconds. However, we were aiming for at least half a million cells and thus built a `CUDA/C++` implementation after three months.